In [5]:
## Together API
import os

os.environ["TOGETHER_API_KEY"] = "93697a53453495a172e4fa73988015204fdafb3b7bf1ad2695eefa214658e57b"

import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': '9356aa066617c5c405f3895fcae5dfc4043528353e5d7e2ac17f57739420afaa'}

In [6]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [7]:
test_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [12]:
test_llm('''You goal is to inspire users to be more aware of undesirable consequences of digital technologies, using insights from the below paper.
Phase 1: Identify the technology; Stop here if you can't identify the technology;
Phase 2: Identify whether the paper discusses an undesirable consequence of the technology (with simple [Yes/No]); Stop here if the answer is No;
Phase 3: Summarize the undesirable consequence of the technology from the paper;
Phase 4: Identify the domain of the technology from this list. List of possible aspects: Health & Well-being, Security & Privacy, Equality & Justice, User Experience, Economy, Access to Information & Discourse, Environment & Sustainability, Politics, Power Dynamics, Social Norms & Relationship.

"Even though today\u2019s videoconferencing systems are often very useful, these systems do not provide support for one of the most important aspects of in-person meetings: the ad hoc, private conversations that happen before, after, and during the breaks of scheduled events\u2013the proverbial hallway conversations. Here we describe our design of a simple system, called Minglr, which supports this kind of interaction by facilitating the matching of conversational partners. We describe two studies of this system\u2019s use at two virtual conferences with over 450 total participants. Our results provide evidence for the usefulness of this capability, showing that, for example, 81% of people who used the system successfully thought that future virtual conferences should include a tool with similar functionality. We believe that similar functionality is likely to be widely implemented in many videoconferencing systems and to increase the feasibility and desirability of many kinds of remote work and socializing."

1.
''')

'Technology: Information and Communication Technologies (ICTs)\n\n2.\nYes\n\n3.\nICTs can be used to support street outreach workers in identifying and mediating conflicts, supporting collaboration and teamwork, and invoking community connections and trust.\n\n4.\nSocial Norms & Relationships'

In [13]:
test_llm('''You goal is to inspire users to be more aware of undesirable consequences of digital technologies, using insights from the below paper.
Phase 1: Identify the technology; Stop here if you can't identify the technology;
Phase 2: Identify whether the paper discusses an undesirable consequence of the technology (with simple [Yes/No]); Stop here if the answer is No;
Phase 3: Summarize the undesirable consequence of the technology from the paper;
Phase 4: Identify the domain of the technology from this list. List of possible aspects: Health & Well-being, Security & Privacy, Equality & Justice, User Experience, Economy, Access to Information & Discourse, Environment & Sustainability, Politics, Power Dynamics, Social Norms & Relationship.

"Even though today\u2019s videoconferencing systems are often very useful, these systems do not provide support for one of the most important aspects of in-person meetings: the ad hoc, private conversations that happen before, after, and during the breaks of scheduled events\u2013the proverbial hallway conversations. Here we describe our design of a simple system, called Minglr, which supports this kind of interaction by facilitating the matching of conversational partners. We describe two studies of this system\u2019s use at two virtual conferences with over 450 total participants. Our results provide evidence for the usefulness of this capability, showing that, for example, 81% of people who used the system successfully thought that future virtual conferences should include a tool with similar functionality. We believe that similar functionality is likely to be widely implemented in many videoconferencing systems and to increase the feasibility and desirability of many kinds of remote work and socializing."

1.
''')

'Technology: Videoconferencing systems\n\n2.\nYes\n\n3.\nLack of support for ad hoc, private conversations that happen during in-person meetings\n\n4.\nUser Experience'

In [14]:
test_llm('''You goal is to inspire users to be more aware of undesirable consequences of digital technologies, using insights from the below paper.
Phase 1: Identify the technology; Stop here if you can't identify the technology;
Phase 2: Identify whether the paper discusses an undesirable consequence of the technology (with simple [Yes/No]); Stop here if the answer is No;
Phase 3: Summarize the undesirable consequence of the technology from the paper;
Phase 4: Identify the domain of the technology from this list. List of possible aspects: Health & Well-being, Security & Privacy, Equality & Justice, User Experience, Economy, Access to Information & Discourse, Environment & Sustainability, Politics, Power Dynamics, Social Norms & Relationship.

"In a corpus of 45,821 app reviews of the top five mobile cryptocurrency wallets, we identified and qualitatively analyzed 6,859 reviews pertaining to the user experience (UX) with those wallets. Our analysis suggests that both new and experienced users struggle with general and domain-specific UX issues that, aside from frustration and disengagement, might lead to dangerous errors and irreversible monetary losses. We reveal shortcomings of current wallet UX as well as users\u2019 misconceptions, some of which can be traced back to a reliance on their understanding of conventional payment systems. For example, some users believed that transactions were free, reversible, and could be canceled anytime, which is not the case in reality. Correspondingly, these beliefs often resulted in unmet expectations. Based on our findings, we provide recommendations on how to design cryptocurrency wallets that both alleviate the identified issues and counteract some of the misconceptions in order to better support newcomers."

1.
''')

'Technology: Mobile cryptocurrency wallets\n\n2.\nYes\n\n3.\nUsers struggle with general and domain-specific UX issues that can lead to dangerous errors and irreversible monetary losses. Users have misconceptions about the technology, such as believing that transactions are free, reversible, and can be canceled anytime.\n\n4.\nUser Experience'

In [88]:
test_llm('''You goal is to inspire users to be more aware of undesirable consequences of digital technologies, using insights from the below paper.
Phase 1: Identify the technology from this list. List of possible technologies: Social Media, or Voice Assistant, or Virtual/Augmented Reality. Return irrelevant and stop here if it is not one of these technologies;
Phase 2: Identify whether the paper discusses an undesirable consequence of the technology (with simple [Yes/No]); Return irrelevant and stop here if the answer is No;
Phase 3: Summarize the undesirable consequence of the technology from the paper for lay audience;
Phase 4: Identify the domain of the technology from this list. List of possible aspects: Health & Well-being, Security & Privacy, Equality & Justice, User Experience, Economy, Access to Information & Discourse, Environment & Sustainability, Politics, Power Dynamics, Social Norms & Relationship.

"Smart energy technologies provide new opportunities for network demand management, while generating data that can provide inference into household activities and which is increasingly valuable for user profiling and targeted marketing. This paper explores the human values implicated by smart energy technologies as energy consumption emerges as a potentially sensitive and attributable data trail. Using a values-based approach involving semi-structured interviews with 39 Australians, we use speculative design to engage participants with the implications of high frequency household energy data and understand users’ values and attitudes towards curtailment, surveillance, accountability and consent. The principal contributions are (1) a nuanced understanding of user values around privacy, responsibility, trust, collectivism and curtailment in relation to interpersonal household energy use data and considerations for speculative designs in this space, and (2) reflections on the design of mechanisms for consumer participation in demand response for energy networks."

Answer:
1. Technology: [Choose from Social Media, or Voice Assistant, or Virtual/Augmented Reality]
2. [Yes/No]
3. [Summary of the undesirable consequence]
4. [Category]

1.
''')

'\n2.\n\n3.\n\n4.'

In [47]:
from langchain import PromptTemplate, LLMChain

template = '''You goal is to inspire users to be more aware of undesirable consequences of digital technologies, using insights from the below paper.
Phase 1: Identify the technology from Social Media, Voice Assistant, Virtual/Augmented Reality; Stop here if you can't identify the technology;
Phase 2: Identify whether the paper discusses an undesirable consequence of the technology (with simple [Yes/No]); Stop here if the answer is No;
Phase 3: Summarize the undesirable consequence of the technology from the paper for lay audience;
Phase 4: Identify the domain of the technology from this list. List of possible aspects: Health & Well-being, Security & Privacy, Equality & Justice, User Experience, Economy, Access to Information & Discourse, Environment & Sustainability, Politics, Power Dynamics, Social Norms & Relationship.

"{abstract}"

1. Technology: Social Media, Voice Assistant, Virtual/Augmented Reality;
2. Yes/No;
3. Summary of the undesirable consequence of the technology for lay audience;
4. Domain: from the list.

1.
'''
prompt = PromptTemplate(template=template, input_variables=["abstract"])
llm_chain = LLMChain(prompt=prompt, llm=test_llm)

In [73]:
import json
import tqdm
with open("chi_abstract_classifier_complete.json", "r") as file:
    output = json.load(file)
    # res = []
    for index, paper in enumerate(tqdm.tqdm(output)):
        if index < 3: continue
        found = False
        for r in res:
            if r["paperId"] == paper["paperId"]:
                found = True
                break
        if found: 
            print(paper)
            continue
        paperId = paper["paperId"]
        title = paper["title"]
        abstract = paper["abstract"]
        answer = llm_chain.run(abstract)
        
        res.append({
            "paperId": paperId,
            "title": title,
            "abstract": abstract,
            "answer": answer,
            "doi": paper["doi"]
        })

  0%|          | 0/7012 [00:00<?, ?it/s]

{'paperId': '49a28d14d6e2526dd717be852ce9c39dd51110bb', 'abstract': 'Smart energy technologies provide new opportunities for network demand management, while generating data that can provide inference into household activities and which is increasingly valuable for user profiling and targeted marketing. This paper explores the human values implicated by smart energy technologies as energy consumption emerges as a potentially sensitive and attributable data trail. Using a values-based approach involving semi-structured interviews with 39 Australians, we use speculative design to engage participants with the implications of high frequency household energy data and understand users’ values and attitudes towards curtailment, surveillance, accountability and consent. The principal contributions are (1) a nuanced understanding of user values around privacy, responsibility, trust, collectivism and curtailment in relation to interpersonal household energy use data and considerations for specul

  5%|▌         | 374/7012 [27:23<8:06:13,  4.39s/it] 


KeyboardInterrupt: 

In [76]:
with open("temp.json", "w") as file:
    json.dump(res, file, indent=4)